# Q1. Embedding the query

In [52]:
import numpy as np
from qdrant_client import models, QdrantClient
from fastembed import TextEmbedding

In [18]:
query = "I just discovered the course. Can I join now?"

In [19]:
model_name = "jinaai/jina-embeddings-v2-small-en"

In [20]:
embedding_model = TextEmbedding(model_name)

In [21]:
q = list(embedding_model.embed(query))[0]
q.min()

np.float64(-0.11726373885183883)

# Q2. Cosine similarity with another vector

In [15]:
doc = 'Can I still join the course after the start date?'

In [16]:
d = list(embedding_model.embed(doc))[0]

In [24]:
q.dot(d)

np.float64(0.9008528895674548)

# Q3. Ranking by cosine

In [25]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [26]:
embs = [
    list(embedding_model.embed(document['text']))[0]
    for document in documents
]

In [31]:
len(documents), len(embs), [len(e) for e in embs]

(5, 5, [512, 512, 512, 512, 512])

In [34]:
V = np.array(embs)
V.shape

(5, 512)

In [39]:
sorted(enumerate(V.dot(q)), key=lambda x: x[1], reverse=True)

[(1, np.float64(0.8182378150042889)),
 (2, np.float64(0.8085397398734188)),
 (0, np.float64(0.7629684696540238)),
 (4, np.float64(0.7304499234333598)),
 (3, np.float64(0.7133079015686243))]

# Q4. Ranking by cosine, version two

In [40]:
embs = [
    list(embedding_model.embed(document['question'] + ' ' + document['text']))[0]
    for document in documents
]

In [41]:
len(documents), len(embs), [len(e) for e in embs]

(5, 5, [512, 512, 512, 512, 512])

In [42]:
V = np.array(embs)
V.shape

(5, 512)

In [43]:
sorted(enumerate(V.dot(q)), key=lambda x: x[1], reverse=True)

[(0, np.float64(0.8514543236908068)),
 (1, np.float64(0.8436594159113069)),
 (2, np.float64(0.840828704850256)),
 (4, np.float64(0.8086007795043939)),
 (3, np.float64(0.7755157969663907))]

It is no longer the document 1, since the question in document 0 is much more similar to the user query than the question in document 1, and this impacts in final ranking.

# Q5. Selecting the embedding model

In [45]:
all_sizes = {model_info['dim'] for model_info in TextEmbedding.list_supported_models()}
all_sizes

{384, 512, 768, 1024}

In [46]:
min(all_sizes)

384

# Q6. Indexing with qdrant

In [48]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [56]:
model_name = 'BAAI/bge-small-en'

In [53]:
client = QdrantClient("http://localhost:6333")

In [54]:
# Define the collection name
collection_name = "zoomcamp-homework-02"

EMBEDDING_DIMENSIONALITY = 384

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [57]:
points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['question'] + ' ' + doc['text'], model=model_name),
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            }
        )
        points.append(point)

        id += 1

In [58]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [59]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_name 
        ),
        limit=limit,
        with_payload=True
    )

    return results

In [63]:
search(query).points[0].score

0.8703172